# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.

Activate it.

You will find more info about virtual environments in the course content and on the web.

## API Scraping

### A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword.

NOTE: external libraries need to be installed first. Check their documentation.

Check the [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the Requests documentation to:
1. use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is the signification of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [1]:
from bs4 import BeautifulSoup
import requests

In [23]:

url = "https://country-leaders.onrender.com/status/"
r = requests.get(url)
print(url, r.status_code)


https://country-leaders.onrender.com/status/ 200


### Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. Check what to do in the Requests [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/).


In [24]:
# Set the countries_url variable (1 line)
root_url = "https://country-leaders.onrender.com"
endpoint = "countries"


# query the /countries en/cookiedpoint using the get() method and store it in the req variable (1 line)
req = requests.get(root_url)
# Get the JSON content and store it in the countries variable (1 line)
countries= requests.get(f"{root_url}/{endpoint}")
# display the request's status code and the countries variable (1 line)
countries.json()

{'message': 'The cookie is missing'}

### Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

{}


In [ ]:
# Set the cookie_url variable (1 line)
def fetch_cookie():
    endpoint="cookie"
    cookie_url=f"{root_url}/{endpoint}"
    # Query the enpoint, set the cookies variable and display it (2 lines)
    cookies1= requests.get(cookie_url)
    cookies1 =cookies1.cookies.get_dict() 
    return cookies1


Try to query the countries endpoint using the cookie, save the output and print it.

In [43]:
# query the /countries endpoint, assign the output to the countries variable (1 line)
endpoint="countries"
countries_url=f"{root_url}/{endpoint}"

countries= requests.get(countries_url,cookies=cookies1)
# display the countries variable (1 line)
countries.json()

['us', 'be', 'ma', 'ru', 'fr']

Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### Getting the actual data from the API

Query the `/leaders` endpoint.

In [54]:
# Set the leaders_url variable (1 line)

endpoint="leaders"
leaders_url=f"{root_url}/{endpoint}"
# query the /leaders endpoint, assign the output to the leaders variable (1 line)
leaders= requests.get(leaders_url,cookies=cookies1)
# display the leaders variable (1 line)
leaders.json()

{'message': 'Please specify a country'}

It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [57]:
# query the /leaders endpoint using cookies and parameters (take any country in countries)
params = {
    "country": "us"}

result= requests.get(leaders_url,cookies=cookies1,params=params)


# assign the output to the leaders variable (1 line)
leaders=result
# display the leaders variable (1 line)
leaders.json()

[{'id': 'Q23',
  'first_name': 'George',
  'last_name': 'Washington',
  'birth_date': '1732-02-22',
  'death_date': '1799-12-14',
  'place_of_birth': 'Westmoreland County',
  'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington',
  'start_mandate': '1789-04-30',
  'end_mandate': '1797-03-04'},
 {'id': 'Q76',
  'first_name': 'Barack',
  'last_name': 'Obama',
  'birth_date': '1961-08-04',
  'death_date': None,
  'place_of_birth': 'Kapiolani Medical Center for Women and Children',
  'wikipedia_url': 'https://en.wikipedia.org/wiki/Barack_Obama',
  'start_mandate': '2009-01-20',
  'end_mandate': '2017-01-20'},
 {'id': 'Q91',
  'first_name': 'Abraham',
  'last_name': 'Lincoln',
  'birth_date': '1809-02-12',
  'death_date': '1865-04-15',
  'place_of_birth': 'Sinking Spring Farm',
  'wikipedia_url': 'https://en.wikipedia.org/wiki/Abraham_Lincoln',
  'start_mandate': '1861-03-04',
  'end_mandate': '1865-04-15'},
 {'id': 'Q207',
  'first_name': 'George',
  'last_name': 'Bush',
  'bi

### A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [68]:
leaders_per_country={}
cookies1=fetch_cookie()
for country in countries.json():
    params= { "country": country}
    result= requests.get(leaders_url,cookies=cookies1,params=params)
    leaders_per_country[country]=result.json()


[{'id': 'Q57553',
  'first_name': 'Mohammed',
  'last_name': 'None',
  'birth_date': '1963-08-21',
  'death_date': None,
  'place_of_birth': 'Rabat',
  'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86',
  'start_mandate': '1999-07-23',
  'end_mandate': None},
 {'id': 'Q69103',
  'first_name': 'Hassan',
  'last_name': 'None',
  'birth_date': '1929-07-09',
  'death_date': '1999-07-20',
  'place_of_birth': 'Rabat',
  'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B3%D9%86_%D8%A7%D9%84%D8%AB%D8%A7%D9%86%D9%8A_%D8%A8%D9%86_%D9%85%D8%AD%D9%85%D8%AF',
  'start_mandate': '1961-02-26',
  'end_mandate': '1999-07-23'},
 {'id': 'Q193874',
  'first_name': 'Mohammed',
  'last_name': 'None',
  'birth_date': '1909-08-10',
  'death_date': '1961-02-26',
  'place_of_birth': 'Fez',
  'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%AE%D8%A7

It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [230]:
# < 15 lines
def get_leaders():

    leaders_per_country={}
    #1 defining urls and endpoints
    root_url = "https://country-leaders.onrender.com"
    countries_endpoint = "countries"
    leaders_endpoint = "leaders"
    #2 Getting cookie
    cookies1=fetch_cookie()
    #3 getting the countires
    countries_url=f"{root_url}/{countries_endpoint}"
    countries= requests.get(countries_url,cookies=cookies1)
    # 4 looping over the countries and geting the reuslts 
    leaders_url=f"{root_url}/{leaders_endpoint}"
    for country in countries.json():
        params= { "country": country}
        result= requests.get(leaders_url,cookies=cookies1,params=params)
        leaders_per_country[country]=result.json()
        
    return leaders_per_country





Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [135]:
# 2 lines
leaders_per_country=get_leaders()
leaders_per_country["ma"]

[{'id': 'Q57553',
  'first_name': 'Mohammed',
  'last_name': 'None',
  'birth_date': '1963-08-21',
  'death_date': None,
  'place_of_birth': 'Rabat',
  'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86',
  'start_mandate': '1999-07-23',
  'end_mandate': None},
 {'id': 'Q69103',
  'first_name': 'Hassan',
  'last_name': 'None',
  'birth_date': '1929-07-09',
  'death_date': '1999-07-20',
  'place_of_birth': 'Rabat',
  'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B3%D9%86_%D8%A7%D9%84%D8%AB%D8%A7%D9%86%D9%8A_%D8%A8%D9%86_%D9%85%D8%AD%D9%85%D8%AF',
  'start_mandate': '1961-02-26',
  'end_mandate': '1999-07-23'},
 {'id': 'Q193874',
  'first_name': 'Mohammed',
  'last_name': 'None',
  'birth_date': '1909-08-10',
  'death_date': '1961-02-26',
  'place_of_birth': 'Fez',
  'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%AE%D8%A7

## Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [153]:
# Accessin Hassan II of Morocco
#Wikipedia_url = next((leader["wikipedia_url"] for leader in leaders_per_country["ma"] if leader["id"] == "Q69103"), None)



result= requests.get(Wikipedia_url,cookies=cookies1)
result.text


'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-disabled skin-theme-clientpref-day vector-toc-available" lang="ar" dir="rtl">\n<head>\n<meta charset="UTF-8">\n<title>الحسن الثاني بن محمد - ويكيبيديا</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpre

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [154]:
from bs4 import BeautifulSoup

cookies1=fetch_cookie()
result = requests.get(Wikipedia_url, cookies=cookies1)
soup = BeautifulSoup(result.text, "html.parser")


# Use prettify to format the HTML and make it readable
pretty_html = soup.prettify()
print(pretty_html)

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-disabled skin-theme-clientpref-day vector-toc-available" dir="rtl" lang="ar">
 <head>
  <meta charset="utf-8"/>
  <title>
   الحسن الثاني بن محمد - ويكيبيديا
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-wi

In [155]:
def get_html(Wikipedia_url):

    
    cookies1=fetch_cookie()
    result = requests.get(Wikipedia_url, cookies=cookies1)
    soup = BeautifulSoup(result.text, "html.parser")
   
    return soup

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [160]:
# 2 lines
paragraphs = soup.find_all('p')
paragraphs

[<p><b>الحَسَن الثاني بِنْ مُحمد بِنْ يوسف العَلوي</b> (9 يوليو 1929 – 23 يوليو 1999)،<sup class="reference" id="cite_ref-archive.org2_12-0"><a href="#cite_note-archive.org2-12"><span class="cite-bracket">[</span>12<span class="cite-bracket">]</span></a></sup> ثاني ملوك المملكة المغربية بعد الإستقلال، والملك الثاني والعشرين للمغرب من <a href="/wiki/%D8%B3%D9%84%D8%A7%D9%84%D8%A9_%D8%A7%D9%84%D8%B9%D9%84%D9%88%D9%8A%D9%8A%D9%86_%D8%A7%D9%84%D9%81%D9%8A%D9%84%D8%A7%D9%84%D9%8A%D9%8A%D9%86_%D8%A7%D9%84%D8%AD%D8%A7%D9%83%D9%85%D8%A9" title="سلالة العلويين الفيلاليين الحاكمة">سلالة العلويين الفيلاليين</a>، تولى حكم المملكة المغربية خلفًا لوالده الملك محمد الخامس في 26 فبراير 1961 وحتى وفاته في 23 يوليو 1999. ينتمي الملك الحسن الثاني إلى السلالة العلوية التي تعود في نسبها إلى <a href="/wiki/%D8%A7%D9%84%D8%AD%D8%B3%D9%86_%D8%A8%D9%86_%D8%B9%D9%84%D9%8A" title="الحسن بن علي">الحسن بن علي بن أبي طالب</a>، وتحكم المغرب منذ عام 1666 ميلادية ويلقب الحاكم منهم بأمير المؤمنين.
 </p>,
 <p>ولد في الر

If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one
  * store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
  * exit the loop

In [161]:


bd=next((leader["birth_date"] for leader in leaders_per_country["ma"] if leader["id"] == "Q69103"), None)
for i, paragraph in enumerate(paragraphs[:5]):  # Display the first 5 paragraphs for inspection
    if bd[:4] in paragraph.get_text():
        print(f"Paragraph {i + 1}:", paragraph.get_text())
        break


Paragraph 1: الحَسَن الثاني بِنْ مُحمد بِنْ يوسف العَلوي (9 يوليو 1929 – 23 يوليو 1999)،[12] ثاني ملوك المملكة المغربية بعد الإستقلال، والملك الثاني والعشرين للمغرب من سلالة العلويين الفيلاليين، تولى حكم المملكة المغربية خلفًا لوالده الملك محمد الخامس في 26 فبراير 1961 وحتى وفاته في 23 يوليو 1999. ينتمي الملك الحسن الثاني إلى السلالة العلوية التي تعود في نسبها إلى الحسن بن علي بن أبي طالب، وتحكم المغرب منذ عام 1666 ميلادية ويلقب الحاكم منهم بأمير المؤمنين.



In [152]:
#printing the first paragraph of Wiki page for each leader for each country in list

for country in countries.json():
    for leader in leaders_per_country[country]:
        Wikipedia_url1=leader["wikipedia_url"]
        soup=get_html(Wikipedia_url1)
        paragraphs = soup.find_all('p')
        start_mandate=leader["start_mandate"][0:4]
        if start_mandate is None:
            continue
        for i, paragraph in enumerate(paragraphs[:5]):  # Display the first 5 paragraphs for inspection4
             if start_mandate in paragraph.get_text():
                 print(f"Paragraph {i + 1}:", paragraph.get_text())
                 break




Paragraph 2: George Washington (February 22, 1732 – December 14, 1799) was a Founding Father of the United States, military officer, and farmer who served as the first president of the United States from 1789 to 1797. Appointed by the Second Continental Congress as commander of the Continental Army in 1775, Washington led Patriot forces to victory in the American Revolutionary War and then served as president of the Constitutional Convention in 1787, which drafted the current Constitution of the United States. Washington has thus become commonly known as the "Father of his Country".

Paragraph 2: Barack Hussein Obama II[a] (born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. As a member of the Democratic Party, he was the first African-American president in U.S. history. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008 and as an Illinois state senator from 1997 to 2004. 

Paragraph 2: A

TypeError: 'NoneType' object is not subscriptable

At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [198]:
 #Function to check if a string contains
def contains_four_digit_number(text):
    for word in text.split():
        if word.isdigit() and len(word) == 4:
            return True
    return False

In [ ]:
import re
# 10 lines
def get_first_paragraph(wikipedia_url):
  print(wikipedia_url) 
  cookies1=fetch_cookie()
  result = requests.get(wikipedia_url, cookies=cookies1)
  soupq = BeautifulSoup(result.text, "html.parser")
  paragraphs1 = soup.find_all('p')
  for i, paragraph in enumerate(paragraphs1):
    first_paragraph = paragraph.get_text()
    if contains_four_digit_number(first_paragraph):
      return first_paragraph
#   return first_paragraph

In [206]:
# Test: 3 lines
url="https://en.wikipedia.org/wiki/George_Washington"
first_paragraph= get_first_paragraph(url)
first_paragraph

https://en.wikipedia.org/wiki/George_Washington


'الحَسَن الثاني بِنْ مُحمد بِنْ يوسف العَلوي (9 يوليو 1929 – 23 يوليو 1999)،[12] ثاني ملوك المملكة المغربية بعد الإستقلال، والملك الثاني والعشرين للمغرب من سلالة العلويين الفيلاليين، تولى حكم المملكة المغربية خلفًا لوالده الملك محمد الخامس في 26 فبراير 1961 وحتى وفاته في 23 يوليو 1999. ينتمي الملك الحسن الثاني إلى السلالة العلوية التي تعود في نسبها إلى الحسن بن علي بن أبي طالب، وتحكم المغرب منذ عام 1666 ميلادية ويلقب الحاكم منهم بأمير المؤمنين.\n'

### Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [184]:
# 3 lines

import re

Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [215]:
# 10 lines

def get_first_paragraph(wikipedia_url):
  print(wikipedia_url) 
  cookies1=fetch_cookie()
  result = requests.get(wikipedia_url, cookies=cookies1)
  soup = BeautifulSoup(result.text, "html.parser")
  paragraphs = soup.find_all('p')
  for i, paragraph in enumerate(paragraphs):
    first_paragraph = paragraph.get_text()
    #searching for the date that come from 4 digits
    
    if re.search(r'\b\d{4}\b',first_paragraph ):
      
    
        return first_paragraph

In [220]:
# Test: 
url="https://en.wikipedia.org/wiki/Barack_Obama"
first_paragraph= get_first_paragraph(url)
first_paragraph

https://en.wikipedia.org/wiki/Barack_Obama


'Barack Hussein Obama II[a] (born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. As a member of the Democratic Party, he was the first African-American president in U.S. history. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008 and as an Illinois state senator from 1997 to 2004. \n'

Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [238]:
# < 20 lines

def get_first_paragraph1(wikipedia_url):
  #print(wikipedia_url) 
  cookies1=fetch_cookie()
  result = requests.get(wikipedia_url, cookies=cookies1)
  soup = BeautifulSoup(result.text, "html.parser")
  paragraphs = soup.find_all('p')
  for i, paragraph in enumerate(paragraphs):
    first_paragraph1 = paragraph.get_text()
    # Pattern to match date formats like Arabic and English dates
    pattern = r"\(([^)]+)\)"
    if re.search(pattern ,first_paragraph1 ):
      return first_paragraph1

In [239]:
# Test: 
url1="https://en.wikipedia.org/wiki/George_Washington"
first_paragraph1= get_first_paragraph1(url1)
print(first_paragraph1)

George Washington (February 22, 1732 – December 14, 1799) was a Founding Father of the United States, military officer, and farmer who served as the first president of the United States from 1789 to 1797. Appointed by the Second Continental Congress as commander of the Continental Army in 1775, Washington led Patriot forces to victory in the American Revolutionary War and then served as president of the Constitutional Convention in 1787, which drafted the current Constitution of the United States. Washington has thus become commonly known as the "Father of his Country".



## Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [254]:
# < 20 lines
#printing the first paragraph of Wiki page for each leader for each country in list
first_paragraph1=""
def get_leaders1():
    first_paragraph1=""
    leaders_per_country={}
    #1 defining urls and endpoints
    root_url = "https://country-leaders.onrender.com"
    countries_endpoint = "countries"
    leaders_endpoint = "leaders"
    #2 Getting cookie
    cookies1=fetch_cookie()
    #3 getting the countires
    countries_url=f"{root_url}/{countries_endpoint}"
    countries= requests.get(countries_url,cookies=cookies1)
    # 4 looping over the countries and geting the reuslts 
    leaders_url=f"{root_url}/{leaders_endpoint}"
    for country in countries.json():
        params= { "country": country}
        result= requests.get(leaders_url,cookies=cookies1,params=params)
        leaders_per_country[country]=result.json()
    for country in countries.json():
        for leader in leaders_per_country[country]:
            Wikipedia_url1=leader["wikipedia_url"]
            first_paragraph1= get_first_paragraph1(Wikipedia_url1)
            leader["first_paragraph"] = first_paragraph1
           
    return leaders_per_country

        



In [259]:
get_leaders1()

{'us': [{'id': 'Q23',
   'first_name': 'George',
   'last_name': 'Washington',
   'birth_date': '1732-02-22',
   'death_date': '1799-12-14',
   'place_of_birth': 'Westmoreland County',
   'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington',
   'start_mandate': '1789-04-30',
   'end_mandate': '1797-03-04',
   'first_paragraph': 'George Washington (February 22, 1732\xa0– December 14, 1799) was a Founding Father of the United States, military officer, and farmer who served as the first president of the United States from 1789 to 1797. Appointed by the Second Continental Congress as commander of the Continental Army in 1775, Washington led Patriot forces to victory in the American Revolutionary War and then served as president of the Constitutional Convention in 1787, which drafted the current Constitution of the United States. Washington has thus become commonly known as the "Father of his Country".\n'},
  {'id': 'Q76',
   'first_name': 'Barack',
   'last_name': 'Obama',
  

In [258]:
# Testing with USA , because testing with the rest can take a lot of times !
#printing the first paragraph of Wiki page for each leader for each country in list
first_paragraph1=""
def get_leadersUS():
    leaders_per_country={}
    #1 defining urls and endpoints
    root_url = "https://country-leaders.onrender.com"
    countries_endpoint = "countries"
    leaders_endpoint = "leaders"
    #2 Getting cookie
    cookies1=fetch_cookie()
    #3 getting the countires
    countries_url=f"{root_url}/{countries_endpoint}"
    countries= requests.get(countries_url,cookies=cookies1)
    # 4 looping over the countries and geting the reuslts 
    leaders_url=f"{root_url}/{leaders_endpoint}"
    params= { "country": "us"}
    result= requests.get(leaders_url,cookies=cookies1,params=params)
    leaders_per_country["us"]=result.json()
    for leader in leaders_per_country["us"]:
            Wikipedia_url1=leader["wikipedia_url"]
            first_paragraph1= get_first_paragraph1(Wikipedia_url1)
            leader["first_paragraph"] = first_paragraph1
           
    return leaders_per_country

        

In [257]:
# Check the output of your function (2 lines)
get_leadersUS()

{'us': [{'id': 'Q23',
   'first_name': 'George',
   'last_name': 'Washington',
   'birth_date': '1732-02-22',
   'death_date': '1799-12-14',
   'place_of_birth': 'Westmoreland County',
   'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington',
   'start_mandate': '1789-04-30',
   'end_mandate': '1797-03-04',
   'first_paragraph': 'George Washington (February 22, 1732\xa0– December 14, 1799) was a Founding Father of the United States, military officer, and farmer who served as the first president of the United States from 1789 to 1797. Appointed by the Second Continental Congress as commander of the Continental Army in 1775, Washington led Patriot forces to victory in the American Revolutionary War and then served as president of the Constitutional Convention in 1787, which drafted the current Constitution of the United States. Washington has thus become commonly known as the "Father of his Country".\n'},
  {'id': 'Q76',
   'first_name': 'Barack',
   'last_name': 'Obama',
  

Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [ ]:
def get_leaders1():
    leaders_per_country = {}
    root_url = "https://country-leaders.onrender.com"
    cookies1 = fetch_cookie()

    # Get countries list
    countries_url = f"{root_url}/countries"
    countries = requests.get(countries_url, cookies=cookies1).json()

    # Fetch leaders for each country and add first Wikipedia paragraph
    leaders_url = f"{root_url}/leaders"
    for country in countries:
        params = {"country": country}
        result = requests.get(leaders_url, cookies=cookies1, params=params)
        
        # Check for cookie expiration and refetch if needed
        if result.status_code in [401, 403]:  
            cookies1 = fetch_cookie()
            result = requests.get(leaders_url, cookies=cookies1, params=params)
        
        leaders_per_country[country] = result.json()
        for leader in leaders_per_country[country]:
            wikipedia_url = leader["wikipedia_url"]
            first_paragraph = get_first_paragraph1(wikipedia_url)
            leader["first_paragraph"] = first_paragraph

    return leaders_per_country


Check the output of your function again.

In [265]:
# Check the output of your function (1 line)
get_leaders1()

{'us': [{'id': 'Q23',
   'first_name': 'George',
   'last_name': 'Washington',
   'birth_date': '1732-02-22',
   'death_date': '1799-12-14',
   'place_of_birth': 'Westmoreland County',
   'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington',
   'start_mandate': '1789-04-30',
   'end_mandate': '1797-03-04',
   'first_paragraph': 'George Washington (February 22, 1732\xa0– December 14, 1799) was a Founding Father of the United States, military officer, and farmer who served as the first president of the United States from 1789 to 1797. Appointed by the Second Continental Congress as commander of the Continental Army in 1775, Washington led Patriot forces to victory in the American Revolutionary War and then served as president of the Constitutional Convention in 1787, which drafted the current Constitution of the United States. Washington has thus become commonly known as the "Father of his Country".\n'},
  {'id': 'Q76',
   'first_name': 'Barack',
   'last_name': 'Obama',
  

Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [267]:
# < 20 lines

def get_first_paragraph_modified(wikipedia_url,session= None):
  
  if session is None:
        session = requests.Session()
        session.cookies.update(fetch_cookie())
  result = requests.get(wikipedia_url, cookies=cookies1)
  soup = BeautifulSoup(result.text, "html.parser")
  paragraphs = soup.find_all('p')
  for i, paragraph in enumerate(paragraphs):
    first_paragraph1 = paragraph.get_text()
    # Pattern to match date formats like Arabic and English dates
    pattern = r"\(([^)]+)\)"
    if re.search(pattern ,first_paragraph1 ):
      return first_paragraph1

In [268]:
get_first_paragraph_modified("https://en.wikipedia.org/wiki/George_Washington")

'George Washington (February 22, 1732\xa0– December 14, 1799) was a Founding Father of the United States, military officer, and farmer who served as the first president of the United States from 1789 to 1797. Appointed by the Second Continental Congress as commander of the Continental Army in 1775, Washington led Patriot forces to victory in the American Revolutionary War and then served as president of the Constitutional Convention in 1787, which drafted the current Constitution of the United States. Washington has thus become commonly known as the "Father of his Country".\n'

Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [271]:
session = requests.Session()


def get_leaders_modified(session):
    leaders_per_country = {}
    root_url = "https://country-leaders.onrender.com"
    cookies1 = fetch_cookie()

    # Get countries list
    countries_url = f"{root_url}/countries"
    countries = requests.get(countries_url, cookies=cookies1).json()

    # Fetch leaders for each country and add first Wikipedia paragraph
    leaders_url = f"{root_url}/leaders"
    for country in countries:
        params = {"country": country}
        result = requests.get(leaders_url, cookies=cookies1, params=params)
        
        # Check for cookie expiration and refetch if needed
        if result.status_code in [401, 403]:  
            cookies1 = fetch_cookie()
            result = requests.get(leaders_url, cookies=cookies1, params=params)
        
        leaders_per_country[country] = result.json()
        for leader in leaders_per_country[country]:
            wikipedia_url = leader["wikipedia_url"]
            first_paragraph = get_first_paragraph_modified(wikipedia_url,session)
            leader["first_paragraph"] = first_paragraph

    return leaders_per_country


Test your new functions.



In [274]:
session1 = requests.Session()
leaders_per_country=get_leaders_modified(session1)


## Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [275]:
# 3 lines
import json
with open("leaders_per_country.json", "w", encoding="utf-8") as f:
    json.dump(leaders_per_country, f, ensure_ascii=False, indent=4)

Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [276]:
# 3 lines
with open("leaders_per_country.json", "r", encoding="utf-8") as f:
            leaders_per_country = json.load(f)
leaders_per_country

{'us': [{'id': 'Q23',
   'first_name': 'George',
   'last_name': 'Washington',
   'birth_date': '1732-02-22',
   'death_date': '1799-12-14',
   'place_of_birth': 'Westmoreland County',
   'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington',
   'start_mandate': '1789-04-30',
   'end_mandate': '1797-03-04',
   'first_paragraph': 'George Washington (February 22, 1732\xa0– December 14, 1799) was a Founding Father of the United States, military officer, and farmer who served as the first president of the United States from 1789 to 1797. Appointed by the Second Continental Congress as commander of the Continental Army in 1775, Washington led Patriot forces to victory in the American Revolutionary War and then served as president of the Constitutional Convention in 1787, which drafted the current Constitution of the United States. Washington has thus become commonly known as the "Father of his Country".\n'},
  {'id': 'Q76',
   'first_name': 'Barack',
   'last_name': 'Obama',
  

Make a function `save(leaders_per_country)` to call this code easily.

In [282]:
# 3 lines
def save (leaders_per_country):
    with open("leaders_per_country.json1", "w", encoding="utf-8") as f:
        json.dump(leaders_per_country, f, ensure_ascii=False, indent=4)
        return



In [283]:
# Call the function (1 line)
save(get_leaders_modified(requests.Session()))


## Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## To go further

If you want to practice scraping, you can read this section and tackle the exercise.

You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!